## ENVIRONMENT VARIABLES

In [1]:
EN_ES_CORPUS_DIR = "/notebooks/master-thesis/corpora/tatoeba-challenge/en-es/" #NOT MODIFIED YET

In [2]:
QUY_ES_CORPUS_DIR = "/notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish"

In [3]:
tokenizer_path = "/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/"

In [4]:
%env TOKENIZER_PATH = /storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en

env: TOKENIZER_PATH=/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en


In [5]:
tokenized_path_es_en = "/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/es-en/"
tokenized_path_quy_es = "/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/quy-es/"

In [6]:
%env TOKENIZED_PATH_ES_EN = /storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/es-en
%env TOKENIZED_PATH_QUY_ES = /storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/quy-es

env: TOKENIZED_PATH_ES_EN=/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/es-en
env: TOKENIZED_PATH_QUY_ES=/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/quy-es


In [7]:
%env BIN_DIR = /storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/bin

env: BIN_DIR=/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/bin


In [8]:
%env MODEL_DIR = /storage/master-thesis/models/caps_quy-es+es-en

env: MODEL_DIR=/storage/master-thesis/models/caps_quy-es+es-en


In [9]:
%env CODE_STORAGE = /storage/code/

env: CODE_STORAGE=/storage/code/


In [10]:
quy_es_en_tokenizer_dir = "/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/"

In [ ]:
! locale

locale: Cannot set LC_CTYPE to default locale: No such file or directory
locale: Cannot set LC_MESSAGES to default locale: No such file or directory
locale: Cannot set LC_ALL to default locale: No such file or directory
LANG=en_US.UTF-8
LANGUAGE=
LC_CTYPE="en_US.UTF-8"
LC_NUMERIC="en_US.UTF-8"
LC_TIME="en_US.UTF-8"
LC_COLLATE="en_US.UTF-8"
LC_MONETARY="en_US.UTF-8"
LC_MESSAGES="en_US.UTF-8"
LC_PAPER="en_US.UTF-8"
LC_NAME="en_US.UTF-8"
LC_ADDRESS="en_US.UTF-8"
LC_TELEPHONE="en_US.UTF-8"
LC_MEASUREMENT="en_US.UTF-8"
LC_IDENTIFICATION="en_US.UTF-8"
LC_ALL=en_US.UTF-8


In [ ]:
! update-locale LANG=en_US.UTF-8 LANGUAGE=en.UTF-8

%env LANG=en_US.UTF-8
%env LC_CTYPE=en_US.UTF-8
%env LC_ALL=en_US.UTF-8

/bin/sh: 1: update-locale: not found
env: LANG=en_US.UTF-8
env: LC_CTYPE=en_US.UTF-8
env: LC_ALL=en_US.UTF-8


In [ ]:
! cat /etc/rc.conf

cat: /etc/rc.conf: No such file or directory


## Libraries

In [15]:
pip install tokenizers==0.10.3

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install fairseq

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Converting tatoeba format to standard format

In [ ]:
import csv

with open(EN_ES_CORPUS_DIR + "original/valid.txt", encoding='utf8') as f:
    es = open(EN_ES_CORPUS_DIR + "dev.es", 'w', encoding='utf8')
    en = open(EN_ES_CORPUS_DIR + "dev.en", 'w', encoding='utf8')
    for line in csv.reader(f, delimiter="\t"):
        es.write(line[3] + "\n")
        en.write(line[2] + "\n")
    en.close()
    es.close()
    
with open(EN_ES_CORPUS_DIR + "original/dev.txt", encoding='utf8') as f:
    es = open(EN_ES_CORPUS_DIR + "train.es", 'w', encoding='utf8')
    en = open(EN_ES_CORPUS_DIR + "train.en", 'w', encoding='utf8')
    for line in csv.reader(f, delimiter="\t"):
        if "\n" in line[2] or "\n" in line[3]:
            continue
        if len(line) != 4:
            continue
            
        es.write(line[3] + "\n")
        en.write(line[2] + "\n")
        
    en.close()
    es.close()
        

### Tokenizing data

In [17]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

en_es_quy_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

In [18]:
from tokenizers.trainers import WordPieceTrainer

en_es_quy_trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"], vocab_size=30000, show_progress=True)

In [19]:
from tokenizers.pre_tokenizers import Whitespace

en_es_quy_tokenizer.pre_tokenizer = Whitespace()

en_files = [f"/notebooks/master-thesis/corpora/tatoeba-challenge/en-es/{split}.en" for split in ["dev", "train"]]
es_files = [f"/notebooks/master-thesis/corpora/tatoeba-challenge/en-es/{split}.es" for split in ["dev", "train"]]
quy_files = [f"/notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/{split}.quy" for split in ["dev", "dict", "train"]]
es2_files = [f"/notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/{split}.es" for split in ["dev", "dict", "train"]]

files = en_files + es_files + es2_files + quy_files
en_es_quy_tokenizer.train(files= files, trainer=en_es_quy_trainer)

In [30]:
from pathlib import Path



def tokenize_files(tokenizer, files, extension, output_path):
    for file in files:
        print(f"Reading file {file}")
        with open(file, encoding='utf8') as f:
          lines = f.readlines()
          tokenized_lines = tokenizer.encode_batch(lines)
          tokenized_name = Path(file).stem
          tokenized_name = output_path + tokenized_name + "." + extension
          print(tokenized_name)
          with open(tokenized_name, 'w', encoding='utf8') as wf:

            wf.writelines([" ".join(t.tokens) + "\n" for t in tokenized_lines])

In [52]:
! mkdir bin

In [43]:
en_es_quy_tokenizer.save(tokenizer_path + "quy-es-en-tokenizer.json")

In [44]:
print(en_es_quy_tokenizer)

In [45]:
tok = en_es_quy_tokenizer.encode("we hold this truth to be self evident that everyone is created equal?")
print(tok.tokens)


['we', 'hold', 'this', 'truth', 'to', 'be', 'self', 'evid', '##ent', 'that', 'everyone', 'is', 'created', 'equal', '?']


In [49]:
tokenize_files(en_es_quy_tokenizer, en_files, "en", tokenized_path_es_en)
tokenize_files(en_es_quy_tokenizer, es_files, "es", tokenized_path_es_en)

tokenize_files(en_es_quy_tokenizer, quy_files, "quy", tokenized_path_quy_es)
tokenize_files(en_es_quy_tokenizer, es2_files, "es", tokenized_path_quy_es)

Reading file /notebooks/master-thesis/corpora/tatoeba-challenge/en-es/dev.en
/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/es-en/dev.en
Reading file /notebooks/master-thesis/corpora/tatoeba-challenge/en-es/train.en
/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/es-en/train.en
Reading file /notebooks/master-thesis/corpora/tatoeba-challenge/en-es/dev.es
/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/es-en/dev.es
Reading file /notebooks/master-thesis/corpora/tatoeba-challenge/en-es/train.es
/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/es-en/train.es
Reading file /notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/dev.quy
/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/quy-es/dev.quy
Reading file /notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/dict.quy
/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/quy-es/dict.quy
Read

In [ ]:
! cat /storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/es-en/dev.en


## Binarizing data

In [65]:
### Removing previous dict files
! rm $BIN_DIR/dict.quy.txt
! rm $BIN_DIR/dict.es.txt
! rm $BIN_DIR/dict.en.txt

In [66]:
## Concatenating all training data
! cat $TOKENIZED_PATH_QUY_ES/train.quy $TOKENIZED_PATH_QUY_ES/train.es $TOKENIZED_PATH_ES_EN/train.es $TOKENIZED_PATH_ES_EN/train.en > $BIN_DIR/train.all

In [67]:
ls

fairseq/  fairseq-dynamic-routing-plugin/


In [68]:
! fairseq-preprocess --source-lang all \
    --trainpref $BIN_DIR/train \
    --destdir $BIN_DIR \
    --workers 20 \
    --only-source

2021-10-15 17:16:11 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/bin', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='all', srcdict=None, target_lang=None, task='translation', tensorboard_logdir=None, testpref=None, tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, t

In [69]:
! fairseq-preprocess --source-lang quy --target-lang es \
    --trainpref $TOKENIZED_PATH_QUY_ES/train --validpref $TOKENIZED_PATH_QUY_ES/dev \
    --destdir $BIN_DIR \
    --srcdict $BIN_DIR/dict.all.txt \
    --tgtdict $BIN_DIR/dict.all.txt \
    --workers 20

2021-10-15 17:16:15 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/bin', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='quy', srcdict='/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/bin/dict.all.txt', target_lang='es', task='translation', tensorboard_lo

### Reusing es dict from quy-es preprocessing

In [70]:
! fairseq-preprocess --source-lang es --target-lang en \
    --trainpref $TOKENIZED_PATH_ES_EN/train --validpref $TOKENIZED_PATH_ES_EN/dev \
    --destdir $BIN_DIR \
    --srcdict $BIN_DIR/dict.all.txt \
    --tgtdict $BIN_DIR/dict.all.txt \
    --workers 20

2021-10-15 17:16:26 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/bin', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='es', srcdict='/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/bin/dict.all.txt', target_lang='en', task='translation', tensorboard_log

## Training model

In [67]:
ls

fairseq/


In [10]:
pip install torch==1.9.1

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [65]:
cd /storage/code

/storage/code


In [78]:
! git clone https://github.com/NVIDIA/apex



Cloning into 'apex'...
remote: Enumerating objects: 8453, done.
remote: Counting objects: 100% (540/540), done.
remote: Compressing objects: 100% (336/336), done.
remote: Total 8453 (delta 307), reused 355 (delta 192), pack-reused 7913
Receiving objects: 100% (8453/8453), 14.38 MiB | 2.87 MiB/s, done.
Resolving deltas: 100% (5706/5706), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:235: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 20.2.4 from /usr/local/lib/python3.6/dist-packages/pip (python 3.6)
Non-user install because site-packages writeable
Created temporary directory: /tmp/pip-ephem-wheel-cache-rkfd15ek
Created temporary directory: /tmp/pip-req-tracker-i_7liawc
Initialized build tracking at /tmp/pip-req-tracker-i_7liawc
Created build tracker: /tmp/pip-req-tracker-i_7liawc
Entered build tracker: /tmp/pip-req-tracker-i_7liawc

In [66]:
cd apex

/storage/code/apex


In [ ]:
pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [29]:
pip uninstall apex

Note: you may need to restart the kernel to use updated packages.


In [28]:
cd .

/storage/code


In [35]:
rm -r ~/.nv

In [36]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [20]:
pip install torchsummary

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [107]:
torch.autograd.set_detect_anomaly(True)

In [53]:
cd /storage/master-thesis/models/caps_quy-es+es-en

/storage/master-thesis/models/caps_quy-es+es-en


In [54]:
rm ./*.pt

In [37]:
rm ./*.pt

In [11]:
cd /storage/code

/storage/code


In [49]:
rm -r fairseq-dynamic-routing-plugin

In [50]:
! git clone https://github.com/eduardosanchezg/fairseq-dynamic-routing-plugin.git

Cloning into 'fairseq-dynamic-routing-plugin'...
remote: Enumerating objects: 32858, done.
remote: Counting objects: 100% (3968/3968), done.
remote: Compressing objects: 100% (1861/1861), done.
remote: Total 32858 (delta 2587), reused 3327 (delta 2018), pack-reused 28890
Receiving objects: 100% (32858/32858), 13.77 MiB | 6.47 MiB/s, done.
Resolving deltas: 100% (24294/24294), done.
Checking out files: 100% (1166/1166), done.


In [11]:
! fairseq-train $BIN_DIR \
    --user-dir $CODE_STORAGE/fairseq-dynamic-routing-plugin/examples/dynamic_routing/dynamic_routing_src/ \
    --arch=capsnet_transformer --share-all-embeddings \
    --task translation_multi_simple_epoch --lang-pairs quy-es,es-en \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --restore-file $MODEL_DIR/checkpoint_last.pt \
    --save-dir $MODEL_DIR/ \
    --keep-last-epochs 2 \
    --reset-optimizer \
    --encoder-langtok "src" \
    --decoder-langtok #\
    #--fp16
    #--update-freq 64
    
    #--fp16-scale-tolerance=0.25


2021-12-14 08:22:21 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='capsnet_transformer', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='/storage/master-thesis/models/caps_quy-es+es-en/tokenizers/quy-es-en/bin', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=8, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=2048, decoder_input_dim=512, decoder_langtok=True, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normaliz

In [32]:
cp *.pt /notebooks

In [19]:
ls

apex/         den.pt    fairseq-dynamic-routing-plugin/  num.pt  vj.pt
checkpoints/  fairseq/  norm.pt                          sj.pt


In [ ]:
cd /storage/master-thesis/models/quy-es+es-en

/storage/master-thesis/models/quy-es+es-en


In [ ]:
! rm checkpoint108.pt
! rm checkpoint109.pt
! rm checkpoint_best.pt
! rm checkpoint_last.pt

In [ ]:
! rm dict.en.txt
! rm dict.es.txt
! rm dict.quy.txt

In [ ]:
! du -shc /notebooks/*

2.5K	/notebooks/CITATION.cff
5.5K	/notebooks/CODE_OF_CONDUCT.md
15K	/notebooks/CONTRIBUTING.md
19K	/notebooks/ISSUES.md
12K	/notebooks/LICENSE
512	/notebooks/MANIFEST.in
3.5K	/notebooks/Makefile
41K	/notebooks/README.md
41K	/notebooks/README_zh-hans.md
42K	/notebooks/README_zh-hant.md
12K	/notebooks/docker
4.6M	/notebooks/docs
5.0M	/notebooks/examples
8.5K	/notebooks/hubconf.py
7.8G	/notebooks/master-thesis
1.5K	/notebooks/model_cards
9.5K	/notebooks/notebooks
512	/notebooks/pyproject.toml
64K	/notebooks/scripts
1.0K	/notebooks/setup.cfg
13K	/notebooks/setup.py
13M	/notebooks/src
731K	/notebooks/templates
4.5K	/notebooks/test quy-es -> es-en model.ipynb
6.8M	/notebooks/tests
147K	/notebooks/train es-en model.ipynb
158K	/notebooks/train quy-es + es-en model.ipynb
160K	/notebooks/utils
3.5K	/notebooks/valohai.yaml
7.9G	total


In [ ]:
! du -shc /notebooks/master-thesis/*

512	/notebooks/master-thesis/Untitled.ipynb
362M	/notebooks/master-thesis/corpora
7.5G	/notebooks/master-thesis/models
7.8G	total


In [ ]:
! du -shc /notebooks/master-thesis/models/*

1.6G	/notebooks/master-thesis/models/es-en
802M	/notebooks/master-thesis/models/quy-es
5.2G	/notebooks/master-thesis/models/quy-es+es-en
7.5G	total


In [ ]:
! du -shc /*

19M	/apex
5.0M	/bin
4.0K	/boot
24K	/content
^C


In [ ]:
ls

 CITATION.cff         docker/          setup.py
 CODE_OF_CONDUCT.md   docs/            src/
 CONTRIBUTING.md      examples/        templates/
 ISSUES.md            hubconf.py      'test quy-es -> es-en model.ipynb'
 LICENSE              master-thesis/   tests/
 MANIFEST.in          model_cards/    'train es-en model.ipynb'
 Makefile             notebooks/      'train quy-es + es-en model.ipynb'
 README.md            pyproject.toml   utils/
 README_zh-hans.md    scripts/         valohai.yaml
 README_zh-hant.md    setup.cfg


/storage/master-thesis
